In [6]:
# ==== 只用 05HW & 14IB：对齐 -> 窗口化 -> 拼接 -> 划分 -> 保存 ====
import os, numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split

DATA_DIR = "data"
FIVE = ['walk','lie','stand','sit','bend']
label_to_id = {s:i for i,s in enumerate(FIVE)}

def load_one(poses_path, labels_path, kept_idx_path):
    X = np.load(poses_path)                         # (T, 32, 3)  —— 你已经做过 center/scale 和去零
    y = np.load(labels_path, allow_pickle=True)     # (T,)
    keep = np.load(kept_idx_path)                   # (K,)
    # 合法化 keep，并对齐三者
    T = len(X)
    keep = keep[(keep >= 0) & (keep < T)]
    X = X[keep]
    y = y[keep]
    # 标签映射：如果已经是 int 0..4 就不动；如果是字符串/None 处理一下
    if y.dtype.kind in ('U','S','O'):
        y = np.array([label_to_id.get(str(v), -1) if v is not None else -1 for v in y], dtype=np.int64)
    else:
        y = y.astype(np.int64, copy=False)
    return X, y  # X:(K,32,3), y:(K,)

def windowize(X, y, win=15, stride=5, strict=False, mode='center'):
    """X:(T,32,3), y:(T,), 返回 Xw:(N,3,win,32), Yw:(N,)"""
    T = len(X)
    starts = range(0, max(0, T - win + 1), stride)
    Xw, Yw = [], []
    for s in starts:
        seg = X[s:s+win]           # (win,32,3)
        labs = y[s:s+win]          # (win,)
        if len(seg) < win:  # 保险：不够长就跳过
            continue
        if strict:
            if np.any(labs < 0):   # 有无效标签
                continue
        if mode == 'center':
            c = labs[win//2]
            if c < 0:              # 中心无效
                continue
            yw = int(c)
        else:  # majority
            votes = labs[labs >= 0]
            if len(votes) == 0:
                continue
            cnt = np.bincount(votes, minlength=5)
            yw = int(cnt.argmax())
        # (win,32,3) -> (3,win,32)
        seg_ctv = np.transpose(seg.astype(np.float32), (2,0,1))
        Xw.append(seg_ctv)
        Yw.append(yw)
    if len(Xw) == 0:
        return np.empty((0,3,win,32), np.float32), np.empty((0,), np.int64)
    return np.stack(Xw, axis=0), np.asarray(Yw, dtype=np.int64)

def prepare_split(name, win=15, stride=5, strict=False, mode='center'):
    x_path = os.path.join(DATA_DIR, f"poses_{name}_cs_nz.npy")
    y_path = os.path.join(DATA_DIR, f"labels_{name}.npy")
    k_path = os.path.join(DATA_DIR, f"poses_{name}_kept_idx.npy")
    X, y = load_one(x_path, y_path, k_path)
    Xw, Yw = windowize(X, y, win=win, stride=stride, strict=strict, mode=mode)
    print(f"[{name}] Xw={Xw.shape}, Yw={Yw.shape}, class_cnt={Counter(Yw.tolist())}")
    return Xw, Yw

# ---- 只用两个新数据 ----
X05, y05 = prepare_split("05HW", win=15, stride=5, strict=False, mode='center')
X14, y14 = prepare_split("14IB", win=15, stride=5, strict=False, mode='center')

# ---- 合并并分层划分 8/1/1 ----
X_all = np.concatenate([X05, X14], axis=0)
y_all = np.concatenate([y05, y14], axis=0)
print("[ALL]", X_all.shape, Counter(y_all.tolist()))

# 先 train/valtest，再把 valtest 对半分
X_tr, X_vt, y_tr, y_vt = train_test_split(X_all, y_all, test_size=0.2, random_state=42, stratify=y_all)
X_va, X_te, y_va, y_te = train_test_split(X_vt, y_vt, test_size=0.5, random_state=42, stratify=y_vt)

print("[SPLIT]")
print(" Train:", X_tr.shape, Counter(y_tr.tolist()))
print(" Val  :", X_va.shape, Counter(y_va.tolist()))
print(" Test :", X_te.shape, Counter(y_te.tolist()))

# ---- 保存为 stgcn 用 npz（meta 里带上 5 类标签顺序）----
os.makedirs(DATA_DIR, exist_ok=True)
np.savez_compressed(os.path.join(DATA_DIR, "train_stgcn.npz"), X=X_tr, Y=y_tr, meta=dict(labels=FIVE))
np.savez_compressed(os.path.join(DATA_DIR, "val_stgcn.npz"),   X=X_va, Y=y_va, meta=dict(labels=FIVE))
np.savez_compressed(os.path.join(DATA_DIR, "test_stgcn.npz"),  X=X_te, Y=y_te, meta=dict(labels=FIVE))
print("[OK] Saved to data/train_stgcn.npz, data/val_stgcn.npz, data/test_stgcn.npz")


[05HW] Xw=(439, 3, 15, 32), Yw=(439,), class_cnt=Counter({3: 229, 2: 155, 4: 55})
[14IB] Xw=(2459, 3, 15, 32), Yw=(2459,), class_cnt=Counter({2: 1584, 0: 766, 3: 58, 4: 51})
[ALL] (2898, 3, 15, 32) Counter({2: 1739, 0: 766, 3: 287, 4: 106})
[SPLIT]
 Train: (2318, 3, 15, 32) Counter({2: 1391, 0: 613, 3: 229, 4: 85})
 Val  : (290, 3, 15, 32) Counter({2: 174, 0: 77, 3: 29, 4: 10})
 Test : (290, 3, 15, 32) Counter({2: 174, 0: 76, 3: 29, 4: 11})
[OK] Saved to data/train_stgcn.npz, data/val_stgcn.npz, data/test_stgcn.npz


In [1]:
import numpy as np
import os

DATASETS = [
    "16GZ",
    "17JP",
    "19MM",
    "09SY",
    "05HW",
    "14IB"
]

DATA_DIR = "data"   # 如果你的 npy 在 data/

def fix_one(name):
    print(f"\n===== 修复 {name} =====")

    pose_path = os.path.join(DATA_DIR, f"poses_{name}.npy")
    keep_path = os.path.join(DATA_DIR, f"poses_{name}_kept_idx.npy")
    label_path = os.path.join(DATA_DIR, f"labels_{name}.npy")

    X = np.load(pose_path)              # (T_pose, 32, 3)
    T_pose = len(X)

    if os.path.exists(keep_path):
        keep = np.load(keep_path)       # (K,)
    else:
        print(f"[WARNING] {name} 没有 kept_idx，跳过")
        return

    # ---- 修复 kept_idx 越界 ----
    bad = keep >= T_pose
    n_bad = bad.sum()

    if n_bad > 0:
        print(f"[fix] {name}: 发现 {n_bad} 个越界 kept_idx，已删除")
        keep_clean = keep[~bad]
        np.save(keep_path, keep_clean)
    else:
        print(f"[OK] {name}: kept_idx 完全合法")

    # ---- 重新统计 label 情况（如果有 label） ----
    if os.path.exists(label_path):
        y = np.load(label_path)
        print(f"标签数量: {len(y)}, pose数量: {T_pose}, 有效率={len(y)/T_pose:.2f}")
    else:
        print("无 label 文件")

    print(f"[完成] {name} 修复完成")


# ============================
# 运行全部数据集
# =====================


In [2]:
import os
import numpy as np

DATASETS = ["16GZ","17JP","19MM","09SY","05HW","14IB"]
DATA_DIR = "data"

print("开始测试打印...")

for name in DATASETS:
    pose_path = os.path.join(DATA_DIR, f"poses_{name}.npy")
    keep_path = os.path.join(DATA_DIR, f"poses_{name}_kept_idx.npy")
    label_path = os.path.join(DATA_DIR, f"labels_{name}.npy")

    print("\n===== ", name, " =====")
    print("pose_path :", pose_path, "存在？", os.path.exists(pose_path))
    print("keep_path :", keep_path, "存在？", os.path.exists(keep_path))
    print("label_path:", label_path, "存在？", os.path.exists(label_path))


开始测试打印...

=====  16GZ  =====
pose_path : data\poses_16GZ.npy 存在？ True
keep_path : data\poses_16GZ_kept_idx.npy 存在？ True
label_path: data\labels_16GZ.npy 存在？ True

=====  17JP  =====
pose_path : data\poses_17JP.npy 存在？ True
keep_path : data\poses_17JP_kept_idx.npy 存在？ True
label_path: data\labels_17JP.npy 存在？ True

=====  19MM  =====
pose_path : data\poses_19MM.npy 存在？ True
keep_path : data\poses_19MM_kept_idx.npy 存在？ True
label_path: data\labels_19MM.npy 存在？ True

=====  09SY  =====
pose_path : data\poses_09SY.npy 存在？ True
keep_path : data\poses_09SY_kept_idx.npy 存在？ True
label_path: data\labels_09SY.npy 存在？ True

=====  05HW  =====
pose_path : data\poses_05HW.npy 存在？ True
keep_path : data\poses_05HW_kept_idx.npy 存在？ True
label_path: data\labels_05HW.npy 存在？ True

=====  14IB  =====
pose_path : data\poses_14IB.npy 存在？ True
keep_path : data\poses_14IB_kept_idx.npy 存在？ True
label_path: data\labels_14IB.npy 存在？ True


In [3]:
import numpy as np
import os
import sys

DATASETS = ["16GZ","17JP","19MM","09SY","05HW","14IB"]
DATA_DIR = "data"

def flush_print(*msg):
    print(*msg)
    sys.stdout.flush()

def fix_one(name):
    flush_print(f"\n===== 修复 {name} =====")

    pose_path = os.path.join(DATA_DIR, f"poses_{name}.npy")
    keep_path = os.path.join(DATA_DIR, f"poses_{name}_kept_idx.npy")
    label_path = os.path.join(DATA_DIR, f"labels_{name}.npy")

    if not os.path.exists(pose_path):
        flush_print(f"[ERR] 缺少 pose: {pose_path}")
        return
    if not os.path.exists(keep_path):
        flush_print(f"[WARN] 缺少 kept_idx: {keep_path}")
        return

    X = np.load(pose_path)
    K = len(X)
    keep = np.load(keep_path)

    flush_print(f"{name}: pose={K}, kept_idx={len(keep)}")

    bad = keep >= K
    n_bad = bad.sum()

    flush_print(f"{name}: bad kept_idx={n_bad}")

    if n_bad > 0:
        keep = keep[~bad]
        np.save(keep_path, keep)
        flush_print(f"[FIXED] 新 kept_idx={len(keep)}")
    else:
        flush_print("[OK] 无越界")

    if os.path.exists(label_path):
        y = np.load(label_path)
        flush_print(f"{name}: labels={len(y)}, valid_rate={len(y)/K:.2f}")
    else:
        flush_print("[WARN] 无 label 文件")


flush_print("======== 开始修复所有数据集 ========\n")

for name in DATASETS:
    fix_one(name)

flush_print("\n✅ 全部数据集已修复完毕")


======== 开始修复所有数据集 ========


===== 修复 16GZ =====
16GZ: pose=33409, kept_idx=16970
16GZ: bad kept_idx=0
[OK] 无越界


ValueError: Object arrays cannot be loaded when allow_pickle=False

In [4]:
import os
import numpy as np
from collections import Counter

DATA_DIR = "data"
DATASETS = ["16GZ", "17JP", "19MM", "09SY", "05HW", "14IB"]
FIVE = ["walk", "lie", "stand", "sit", "bend"]
MAP = {"walk":0, "lie":1, "stand":2, "sit":3, "bend":4}

def load_np(path):
    """自动 fallback → allow_pickle=True"""
    try:
        return np.load(path)
    except:
        return np.load(path, allow_pickle=True)

def load_pose(name):
    return load_np(os.path.join(DATA_DIR, f"poses_{name}_cs_nz.npy"))

def load_label(name):
    return load_np(os.path.join(DATA_DIR, f"labels_{name}.npy"))

def load_keep(name):
    return load_np(os.path.join(DATA_DIR, f"poses_{name}_kept_idx.npy"))

def map_label_to_five(raw_list):
    """将字符串标签映射到 0-4 类"""
    out = []
    for v in raw_list:
        if v in MAP:
            out.append(MAP[v])
        else:
            out.append(None)
    return np.array(out, dtype=object)

def filter_kept(X, y, keep):
    """根据 kept_idx 过滤干净序列"""
    keep = keep.astype(int)
    keep = keep[(keep >= 0) & (keep < len(X))]
    return X[keep], y[keep]

def window_center(X, y, win=15, stride=5):
    """窗口化（中心帧决定标签）"""
    N = len(X)
    xs, ys = [], []
    for i in range(0, N-win+1, stride):
        seg = X[i:i+win]     # (15, 32, 3)
        lbs = y[i:i+win]
        lbs = [v for v in lbs if v is not None]
        if len(lbs)==0: continue
        label = lbs[len(lbs)//2]      # center label
        xs.append(seg)
        ys.append(label)
    Xw = np.stack(xs).transpose(0, 3, 1, 2)  # → (Nw, 3, 15, 32)
    Yw = np.array(ys, dtype=int)
    return Xw, Yw

# ================================================================
# ✅ Part 1: 单数据集完全处理
# ================================================================
def process_one(name):
    print(f"\n===== {name} =====")
    X = load_pose(name)                        # (T, 32, 3)
    y_raw = load_label(name)                   # (T,)
    keep = load_keep(name)                     # (K,)
    
    y = map_label_to_five(y_raw)               # 映射五类
    X, y = filter_kept(X, y, keep)             # NDF 过滤

    print(f"{name}: after keep → X={X.shape}, y={len(y)}, class_cnt={Counter(y[y!=None])}")

    Xw, Yw = window_center(X, y)
    print(f"{name}: window → Xw={Xw.shape}, Yw={Yw.shape}, class_cnt={Counter(Yw)}")
    return Xw, Yw

# ================================================================
# ✅ Part 2: 合并全部数据集
# ================================================================
def combine_all():
    XX, YY = [], []
    for name in DATASETS:
        Xw, Yw = process_one(name)
        XX.append(Xw)
        YY.append(Yw)
    X = np.concatenate(XX, axis=0)
    Y = np.concatenate(YY, axis=0)
    print("\n===== ALL merged =====")
    print(f"X={X.shape}, Y={Y.shape}, class_cnt={Counter(Y)}")
    return X, Y

# ================================================================
# ✅ Part 3: 自动 oversample 少数类
# ================================================================
def oversample(X, Y):
    print("\n===== Oversample =====")
    cnt = Counter(Y)
    M = max(cnt.values())  # 最大类作为目标数量
    XX, YY = [], []
    for c in range(5):
        idx = np.where(Y==c)[0]
        if len(idx)==0: continue
        if len(idx)<M:
            rep = np.random.choice(idx, M-len(idx), replace=True)
            idx = np.concatenate([idx, rep])
        XX.append(X[idx])
        YY.append(np.full(len(idx), c))
    Xb = np.concatenate(XX)
    Yb = np.concatenate(YY)
    print(f"After oversample: {Counter(Yb)}")
    return Xb, Yb

# ================================================================
# ✅ Part 4: stratified train/val/test split
# ================================================================
def split_stratified(X, Y, tr=0.7, va=0.15, seed=123):
    np.random.seed(seed)
    train_idx = []
    val_idx = []
    test_idx = []
    for c in range(5):
        idx = np.where(Y==c)[0]
        np.random.shuffle(idx)
        n = len(idx)
        t1 = int(n*tr)
        t2 = int(n*(tr+va))
        train_idx.extend(idx[:t1])
        val_idx.extend(idx[t1:t2])
        test_idx.extend(idx[t2:])
    return (X[train_idx], Y[train_idx]), (X[val_idx], Y[val_idx]), (X[test_idx], Y[test_idx])

# ================================================================
# ✅ Part 5: RUN ALL
# ================================================================
X, Y = combine_all()
Xb, Yb = oversample(X, Y)
(trainX, trainY), (valX, valY), (testX, testY) = split_stratified(Xb, Yb)

print("\n===== FINAL =====")
print("Train:", trainX.shape, Counter(trainY))
print("Val  :", valX.shape, Counter(valY))
print("Test :", testX.shape, Counter(testY))

np.savez(os.path.join(DATA_DIR, "train_stgcn.npz"), X=trainX, Y=trainY)
np.savez(os.path.join(DATA_DIR, "val_stgcn.npz"),   X=valX, Y=valY)
np.savez(os.path.join(DATA_DIR, "test_stgcn.npz"),  X=testX, Y=testY)

print("\n✅ DONE! 全流程完成！")



===== 16GZ =====
16GZ: after keep → X=(9797, 32, 3), y=9797, class_cnt=Counter({2: 4948, 0: 1694, 3: 1104, 4: 184})
16GZ: window → Xw=(1662, 3, 15, 32), Yw=(1662,), class_cnt=Counter({np.int64(2): 1013, np.int64(0): 356, np.int64(3): 234, np.int64(4): 59})

===== 17JP =====
17JP: after keep → X=(14189, 32, 3), y=14189, class_cnt=Counter({2: 2668, 0: 2549, 3: 784, 4: 384})
17JP: window → Xw=(1442, 3, 15, 32), Yw=(1442,), class_cnt=Counter({np.int64(2): 587, np.int64(0): 554, np.int64(3): 174, np.int64(4): 127})

===== 19MM =====
19MM: after keep → X=(7654, 32, 3), y=7654, class_cnt=Counter({2: 2950, 0: 2291, 3: 503, 4: 385, 1: 4})
19MM: window → Xw=(1331, 3, 15, 32), Yw=(1331,), class_cnt=Counter({np.int64(2): 628, np.int64(0): 470, np.int64(4): 122, np.int64(3): 109, np.int64(1): 2})

===== 09SY =====
09SY: after keep → X=(11520, 32, 3), y=11520, class_cnt=Counter({2: 2752, 0: 489})
09SY: window → Xw=(651, 3, 15, 32), Yw=(651,), class_cnt=Counter({np.int64(2): 554, np.int64(0): 97})



In [5]:
data = np.load("data/train_stgcn.npz")
X_train, Y_train = data["X"], data["Y"]


In [6]:
data = np.load("data/train_stgcn.npz", allow_pickle=True)

print("Keys in file:", data.files)

X_train = data["X"]
Y_train = data["Y"]

print("Shapes:", X_train.shape, Y_train.shape)


Keys in file: ['X', 'Y']
Shapes: (16205, 3, 15, 32) (16205,)


In [3]:
# === ST-GCN 五类训练：读取npz / 训练 / 评估 / 可视化（混淆矩阵 & PR曲线）===

import os, json, time, csv, math
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import (
    f1_score, classification_report, confusion_matrix,
    precision_recall_fscore_support
)
import matplotlib.pyplot as plt

# 你的管线模块（已在项目里）
import stgcn_fiveclass_pipeline as P
STGCN              = P.STGCN
Graph              = P.Graph
FIVE_LABELS        = P.FIVE_LABELS  # ['walk','lie','stand','sit','bend']

# -------------------
# 配置
# -------------------
DATA_DIR   = "data"
TRAIN_NPZ  = os.path.join(DATA_DIR, "train_stgcn.npz")
VAL_NPZ    = os.path.join(DATA_DIR, "val_stgcn.npz")
TEST_NPZ   = os.path.join(DATA_DIR, "test_stgcn.npz")

BATCH_SIZE = 32
EPOCHS     = 80
LR         = 5e-4
PATIENCE   = 12
SAVE_DIR   = DATA_DIR
BEST_PATH  = os.path.join(SAVE_DIR, "best_stgcn.pth")
LOG_CSV    = os.path.join(SAVE_DIR, "train_log.csv")

os.makedirs(SAVE_DIR, exist_ok=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# -------------------
# 数据读入与 DataLoader
# -------------------
def load_npz(path):
    d = np.load(path, allow_pickle=True)
    return d["X"], d["Y"]

X_tr, Y_tr = load_npz(TRAIN_NPZ)
X_va, Y_va = load_npz(VAL_NPZ)
X_te, Y_te = load_npz(TEST_NPZ)

print("Train:", X_tr.shape, Y_tr.shape)
print("Val  :", X_va.shape, Y_va.shape)
print("Test :", X_te.shape, Y_te.shape)

class NPZDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X   # (N, 3, T, V)
        self.Y = Y   # (N,)
    def __len__(self): return len(self.Y)
    def __getitem__(self, i):
        x = torch.tensor(self.X[i], dtype=torch.float32)  # (3,T,V)
        y = torch.tensor(int(self.Y[i]), dtype=torch.long)
        return x, y

train_loader = DataLoader(NPZDataset(X_tr, Y_tr), batch_size=BATCH_SIZE, shuffle=True,  drop_last=False)
val_loader   = DataLoader(NPZDataset(X_va, Y_va), batch_size=BATCH_SIZE, shuffle=False, drop_last=False)
test_loader  = DataLoader(NPZDataset(X_te, Y_te), batch_size=BATCH_SIZE, shuffle=False, drop_last=False)

# -------------------
# 图（K=1 合并子图）
# -------------------
A = np.asarray(Graph().A, dtype=np.float32)
if A.ndim == 2:
    A = A[None, ...]
elif A.ndim == 3 and A.shape[0] != 1:
    A = A.sum(axis=0, keepdims=True)  # 合并为单图
print("Adjacency:", A.shape)

# -------------------
# 通道适配 & GCN块输入自动对齐（避免 channel mismatch）
# -------------------
class InputAdapter(nn.Module):
    # 将输入(3,T,V)投影到16通道
    def __init__(self, in_ch=3, out_ch=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_ch)
        )
    def forward(self, x):  # (N,3,T,V) -> (N,16,T,V)
        return self.net(x)

def _replicate_A_for_gcn(A_np, num_subset):
    if A_np.ndim == 2:
        A_np = A_np[None, ...]
    if A_np.shape[0] != num_subset:
        A_np = np.repeat(A_np, num_subset, axis=0)
    return torch.tensor(A_np, dtype=torch.float32)

def _set_conv_in(conv2d: nn.Conv2d, in_ch: int) -> nn.Conv2d:
    newc = nn.Conv2d(
        in_channels=in_ch,
        out_channels=conv2d.out_channels,
        kernel_size=conv2d.kernel_size,
        stride=conv2d.stride,
        padding=conv2d.padding,
        dilation=conv2d.dilation,
        groups=conv2d.groups,
        bias=(conv2d.bias is not None),
    )
    return newc

def _block_out_channels(gcn_module) -> int:
    # 以第一支conv的 out_channels 作为该块输出通道
    return int(gcn_module.conv_d[0].out_channels)

adapter    = InputAdapter(3, 16).to(device)
core_model = STGCN(in_channels=16, num_class=len(FIVE_LABELS), A=A).to(device)

# A 复制到每个GCN分支
for m in core_model.modules():
    if m.__class__.__name__ == "GCN" and hasattr(m, "A") and hasattr(m, "num_subset"):
        m.A = _replicate_A_for_gcn(A, m.num_subset).to(device)

# 自动对齐 g2/g3/g4 conv_d 的输入通道
blocks = [getattr(core_model, n) for n in ("g1","g2","g3","g4") if hasattr(core_model, n)]
prev_out = _block_out_channels(blocks[0])
for blk in blocks[1:]:
    newlist = []
    for conv in blk.conv_d:
        newlist.append(_set_conv_in(conv, prev_out).to(device))
    blk.conv_d = nn.ModuleList(newlist)
    prev_out = _block_out_channels(blk)

class WrappedSTGCN(nn.Module):
    def __init__(self, adapter, core):
        super().__init__()
        self.adapter = adapter
        self.core    = core
    def forward(self, x):
        x = self.adapter(x)      # (N,3,T,V)->(N,16,T,V)
        return self.core(x)

model = WrappedSTGCN(adapter, core_model).to(device)

# -------------------
# 类别权重 & 优化器
# -------------------
from collections import Counter
cnt = Counter(Y_tr.tolist()); tot = sum(cnt.values())
weights = np.array([tot / max(cnt.get(i,1),1) for i in range(len(FIVE_LABELS))], dtype=np.float32)
weights = weights / weights.mean()
print("Class weights:", np.round(weights,2).tolist())

criterion = nn.CrossEntropyLoss(weight=torch.tensor(weights, dtype=torch.float32, device=device))
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# -------------------
# 评估/可视化
# -------------------
@torch.no_grad()
def eval_loader(loader):
    model.eval()
    y_true, y_pred = [], []
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        out = model(xb)
        y_true.append(yb.cpu().numpy())
        y_pred.append(out.argmax(1).cpu().numpy())
    y_true = np.concatenate(y_true)
    y_pred = np.concatenate(y_pred)
    f1 = f1_score(y_true, y_pred, average="macro", zero_division=0)
    return f1, y_true, y_pred

def plot_confusion(cm, labels, save_path):
    fig, ax = plt.subplots(figsize=(6,5))
    im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels=labels, yticklabels=labels,
           ylabel='True label', xlabel='Predicted label',
           title='Confusion Matrix')
    plt.setp(ax.get_xticklabels(), rotation=30, ha="right", rotation_mode="anchor")
    fmt = 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    fig.savefig(save_path, dpi=120)
    plt.close(fig)

def plot_pr_bars(y_true, y_pred, labels, save_path):
    # 画每类 precision/recall 柱状图（简单直观）
    prec, rec, f1, sup = precision_recall_fscore_support(
        y_true, y_pred, labels=list(range(len(labels))), zero_division=0
    )
    x = np.arange(len(labels))
    w = 0.35
    fig, ax = plt.subplots(figsize=(7,4))
    ax.bar(x - w/2, prec, width=w, label='Precision')
    ax.bar(x + w/2, rec,  width=w, label='Recall')
    for i,v in enumerate(prec):
        ax.text(i - w/2, v+0.01, f"{v:.2f}", ha='center', va='bottom', fontsize=9)
    for i,v in enumerate(rec):
        ax.text(i + w/2, v+0.01, f"{v:.2f}", ha='center', va='bottom', fontsize=9)
    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation=20)
    ax.set_ylim(0,1.05)
    ax.set_title("Per-class Precision / Recall")
    ax.legend()
    fig.tight_layout()
    fig.savefig(save_path, dpi=120)
    plt.close(fig)

# -------------------
# 训练（早停 + 日志）
# -------------------
best_val, best_state, wait = -1.0, None, 0
with open(LOG_CSV, 'w', newline='', encoding='utf-8') as f:
    w = csv.writer(f); w.writerow(["epoch","train_f1","val_f1","test_f1","lr"])

for ep in range(1, EPOCHS+1):
    model.train()
    yT, yP = [], []
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        out = model(xb)
        loss = criterion(out, yb)
        optimizer.zero_grad(); loss.backward(); optimizer.step()
        yT.append(yb.detach().cpu().numpy())
        yP.append(out.argmax(1).detach().cpu().numpy())
    yT = np.concatenate(yT); yP = np.concatenate(yP)
    tr_f1 = f1_score(yT, yP, average="macro", zero_division=0)

    va_f1, _, _ = eval_loader(val_loader)
    te_f1, _, _ = eval_loader(test_loader)
    print(f"Epoch {ep:02d}/{EPOCHS} | Train F1: {tr_f1:.4f} | Val F1: {va_f1:.4f} | Test F1: {te_f1:.4f}")

    # 写日志
    with open(LOG_CSV, 'a', newline='', encoding='utf-8') as f:
        csv.writer(f).writerow([ep, f"{tr_f1:.6f}", f"{va_f1:.6f}", f"{te_f1:.6f}", optimizer.param_groups[0]['lr']])

    # 早停
    if va_f1 > best_val:
        best_val = va_f1
        best_state = {k:v.cpu() for k,v in model.state_dict().items()}
        torch.save(best_state, BEST_PATH)
        wait = 0
    else:
        wait += 1
        if wait >= PATIENCE:
            print("⚠️ Early stop")
            break

# -------------------
# 加载最优 & 最终评估与可视化
# -------------------
if best_state is None and os.path.exists(BEST_PATH):
    best_state = torch.load(BEST_PATH, map_location="cpu")
if best_state is not None:
    model.load_state_dict(best_state)

te_f1, y_true, y_pred = eval_loader(test_loader)
print("\n🎯 Test macro-F1:", round(te_f1,4))
names = [f"a_{n}" for n in FIVE_LABELS]
print("\nClassification report:")
print(classification_report(y_true, y_pred, labels=list(range(len(FIVE_LABELS))),
                           target_names=names, digits=2, zero_division=0))

cm = confusion_matrix(y_true, y_pred, labels=list(range(len(FIVE_LABELS))))
plot_confusion(cm, names, os.path.join(SAVE_DIR, "confusion_matrix.png"))
plot_pr_bars(y_true, y_pred, names, os.path.join(SAVE_DIR, "per_class_pr.png"))
print(f"\n✅ Saved: {BEST_PATH}")
print(f"✅ Logs:  {LOG_CSV}")
print(f"✅ Plots: {os.path.join(SAVE_DIR, 'confusion_matrix.png')}, {os.path.join(SAVE_DIR, 'per_class_pr.png')}")


Device: cpu
Train: (16205, 3, 15, 32) (16205,)
Val  : (1236, 3, 15, 32) (1236,)
Test : (1240, 3, 15, 32) (1240,)
Adjacency: (1, 32, 32)
Class weights: [1.0, 1.0, 1.0, 1.0, 1.0]
Epoch 01/80 | Train F1: 0.5131 | Val F1: 0.3576 | Test F1: 0.2555
Epoch 02/80 | Train F1: 0.5968 | Val F1: 0.4034 | Test F1: 0.5082
Epoch 03/80 | Train F1: 0.6433 | Val F1: 0.3759 | Test F1: 0.3470
Epoch 04/80 | Train F1: 0.6755 | Val F1: 0.3994 | Test F1: 0.3036
Epoch 05/80 | Train F1: 0.7040 | Val F1: 0.4897 | Test F1: 0.3848
Epoch 06/80 | Train F1: 0.7305 | Val F1: 0.5019 | Test F1: 0.3930
Epoch 07/80 | Train F1: 0.7467 | Val F1: 0.5409 | Test F1: 0.4241
Epoch 08/80 | Train F1: 0.7675 | Val F1: 0.5428 | Test F1: 0.4439
Epoch 09/80 | Train F1: 0.7857 | Val F1: 0.5536 | Test F1: 0.4569
Epoch 10/80 | Train F1: 0.7950 | Val F1: 0.5744 | Test F1: 0.4761
Epoch 11/80 | Train F1: 0.8054 | Val F1: 0.4583 | Test F1: 0.4488
Epoch 12/80 | Train F1: 0.8177 | Val F1: 0.5993 | Test F1: 0.4966
Epoch 13/80 | Train F1: 0.8253 

In [2]:
import os
import numpy as np
from collections import Counter

DATA_DIR = "data"
DATASETS = ["16GZ", "17JP", "19MM", "09SY", "05HW", "14IB"]
FIVE = ["walk", "lie", "stand", "sit", "bend"]
MAP = {"walk":0, "lie":1, "stand":2, "sit":3, "bend":4}

# ------------------------------------------------
# 工具函数
# ------------------------------------------------
def load_np(path):
    """自动 fallback → allow_pickle=True"""
    try:
        return np.load(path)
    except:
        return np.load(path, allow_pickle=True)

def load_pose(name):
    return load_np(os.path.join(DATA_DIR, f"poses_{name}_cs_nz.npy"))

def load_label(name):
    return load_np(os.path.join(DATA_DIR, f"labels_{name}.npy"))

def load_keep(name):
    return load_np(os.path.join(DATA_DIR, f"poses_{name}_kept_idx.npy"))

def map_label_to_five(raw_list):
    """将字符串标签映射到 0-4 类，其他设为 None"""
    out = []
    for v in raw_list:
        if v in MAP:
            out.append(MAP[v])
        else:
            out.append(None)
    return np.array(out, dtype=object)

def filter_kept(X, y, keep):
    """根据 kept_idx 过滤干净序列"""
    keep = keep.astype(int)
    keep = keep[(keep >= 0) & (keep < len(X))]
    return X[keep], y[keep]

def window_center(X, y, win=15, stride=5):
    """窗口化（中心帧决定标签；忽略窗口中全是 None 的情况）"""
    N = len(X)
    xs, ys = [], []
    for i in range(0, N-win+1, stride):
        seg = X[i:i+win]     # (win, 32, 3)
        lbs = y[i:i+win]
        lbs = [v for v in lbs if v is not None]
        if len(lbs) == 0:
            continue
        label = lbs[len(lbs)//2]      # 这里是“有效标签中的中心”
        xs.append(seg)
        ys.append(label)
    if len(xs) == 0:
        return None, None
    Xw = np.stack(xs).transpose(0, 3, 1, 2)  # (Nw, 3, win, 32)
    Yw = np.array(ys, dtype=int)
    return Xw, Yw

# ================================================================
# ✅ Part 1: 单数据集完全处理
# ================================================================
def process_one(name):
    print(f"\n===== {name} =====")
    X = load_pose(name)                        # (T, 32, 3)
    y_raw = load_label(name)                   # (T,)
    keep = load_keep(name)                     # (K,)
    
    y = map_label_to_five(y_raw)               # 映射到五类 / None
    X, y = filter_kept(X, y, keep)             # NDF 过滤
    
    valid_mask = np.array([v is not None for v in y])
    print(f"{name}: after keep → X={X.shape}, y={len(y)}, valid_frames={valid_mask.sum()}, "
          f"class_cnt={Counter(y[valid_mask])}")

    Xw, Yw = window_center(X, y)
    if Xw is None:
        print(f"{name}: no valid windows, skip.")
        return None, None
    print(f"{name}: window → Xw={Xw.shape}, Yw={Yw.shape}, class_cnt={Counter(Yw)}")
    return Xw, Yw

# ================================================================
# ✅ Part 2: 合并全部数据集（窗口后的 X,Y）
# ================================================================
def combine_all():
    XX, YY = [], []
    for name in DATASETS:
        Xw, Yw = process_one(name)
        if Xw is None:
            continue
        XX.append(Xw)
        YY.append(Yw)
    X = np.concatenate(XX, axis=0)
    Y = np.concatenate(YY, axis=0)
    print("\n===== ALL merged (windowed) =====")
    print(f"X={X.shape}, Y={Y.shape}, class_cnt={Counter(Y)}")
    return X, Y

# ================================================================
# ✅ Part 3: stratified train/val/test split  （先 split，再 oversample）
# ================================================================
def split_stratified(X, Y, tr=0.7, va=0.15, seed=123):
    """按类别分层划分 train / val / test（不做 oversample）"""
    np.random.seed(seed)
    train_idx, val_idx, test_idx = [], [], []
    for c in range(5):
        idx = np.where(Y == c)[0]
        if len(idx) == 0:
            continue
        np.random.shuffle(idx)
        n = len(idx)
        t1 = int(n * tr)
        t2 = int(n * (tr + va))
        train_idx.extend(idx[:t1])
        val_idx.extend(idx[t1:t2])
        test_idx.extend(idx[t2:])
    train_idx = np.array(train_idx, dtype=int)
    val_idx   = np.array(val_idx, dtype=int)
    test_idx  = np.array(test_idx, dtype=int)
    return (X[train_idx], Y[train_idx]), (X[val_idx], Y[val_idx]), (X[test_idx], Y[test_idx])

# ================================================================
# ✅ Part 4: oversample 只对训练集进行
# ================================================================
def oversample_train(X, Y, seed=123):
    """
    仅对训练集做类内 oversample，目标：把每一类补到最大类数量。
    注意：千万不要在合并全部数据后再 oversample 否则会 data leak。
    """
    np.random.seed(seed)
    print("\n===== Oversample (TRAIN ONLY) =====")
    cnt = Counter(Y)
    print("Before oversample (train):", cnt)
    M = max(cnt.values())   # 目标：补到最大类数量

    XX, YY = [], []
    for c in range(5):
        idx = np.where(Y == c)[0]
        if len(idx) == 0:
            continue
        if len(idx) < M:
            rep = np.random.choice(idx, M - len(idx), replace=True)
            idx = np.concatenate([idx, rep])
        XX.append(X[idx])
        YY.append(np.full(len(idx), c, dtype=int))
    Xb = np.concatenate(XX, axis=0)
    Yb = np.concatenate(YY, axis=0)
    print("After  oversample (train):", Counter(Yb))
    return Xb, Yb

# ================================================================
# ✅ Part 5: RUN ALL （无 data leak 版本）
# ================================================================
X, Y = combine_all()

# 1) 先用原始窗口数据做分层划分
(trainX_raw, trainY_raw), (valX, valY), (testX, testY) = split_stratified(X, Y)

print("\n===== SPLIT (before oversample) =====")
print("Train raw:", trainX_raw.shape, Counter(trainY_raw))
print("Val      :", valX.shape,      Counter(valY))
print("Test     :", testX.shape,     Counter(testY))

# 2) 只对训练集 oversample，验证/测试保持真实分布
trainX, trainY = oversample_train(trainX_raw, trainY_raw)

print("\n===== FINAL =====")
print("Train (oversampled):", trainX.shape, Counter(trainY))
print("Val               :", valX.shape,   Counter(valY))
print("Test              :", testX.shape,  Counter(testY))

# 3) 保存 —— 训练集用 oversampled 版本；val/test 用原始版本
np.savez(os.path.join(DATA_DIR, "train_stgcn.npz"), X=trainX,      Y=trainY)
np.savez(os.path.join(DATA_DIR, "val_stgcn.npz"),   X=valX,        Y=valY)
np.savez(os.path.join(DATA_DIR, "test_stgcn.npz"),  X=testX,       Y=testY)

print("\n✅ DONE! 无 data leak 版本已完成，train/val/test 均已保存。")



===== 16GZ =====
16GZ: after keep → X=(9797, 32, 3), y=9797, valid_frames=7930, class_cnt=Counter({2: 4948, 0: 1694, 3: 1104, 4: 184})
16GZ: window → Xw=(1662, 3, 15, 32), Yw=(1662,), class_cnt=Counter({np.int64(2): 1013, np.int64(0): 356, np.int64(3): 234, np.int64(4): 59})

===== 17JP =====
17JP: after keep → X=(14189, 32, 3), y=14189, valid_frames=6385, class_cnt=Counter({2: 2668, 0: 2549, 3: 784, 4: 384})
17JP: window → Xw=(1442, 3, 15, 32), Yw=(1442,), class_cnt=Counter({np.int64(2): 587, np.int64(0): 554, np.int64(3): 174, np.int64(4): 127})

===== 19MM =====
19MM: after keep → X=(7654, 32, 3), y=7654, valid_frames=6133, class_cnt=Counter({2: 2950, 0: 2291, 3: 503, 4: 385, 1: 4})
19MM: window → Xw=(1331, 3, 15, 32), Yw=(1331,), class_cnt=Counter({np.int64(2): 628, np.int64(0): 470, np.int64(4): 122, np.int64(3): 109, np.int64(1): 2})

===== 09SY =====
09SY: after keep → X=(11520, 32, 3), y=11520, valid_frames=3241, class_cnt=Counter({2: 2752, 0: 489})
09SY: window → Xw=(651, 3, 